In [1]:
!pip install openai-whisper
!pip install whisperx
!pip install TorchCRF
!pip3 install -U funasr

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 803.2/803.2 kB 11.3 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 132.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 103.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 58.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 4.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 45.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 20.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 100.1 

In [2]:
import torch
import torch.nn as nn
from torch.utils.data import DataLoader, Dataset
import torchaudio
from whisper import load_model
import whisper
import numpy as np
import whisperx
from funasr import AutoModel

Encoder_DIM = 512

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
whisper_model = load_model(name='base', download_root='./').to(device)
whisper_model.eval()

100%|███████████████████████████████████████| 139M/139M [00:01<00:00, 82.9MiB/s]


Whisper(
  (encoder): AudioEncoder(
    (conv1): Conv1d(80, 512, kernel_size=(3,), stride=(1,), padding=(1,))
    (conv2): Conv1d(512, 512, kernel_size=(3,), stride=(2,), padding=(1,))
    (blocks): ModuleList(
      (0-5): 6 x ResidualAttentionBlock(
        (attn): MultiHeadAttention(
          (query): Linear(in_features=512, out_features=512, bias=True)
          (key): Linear(in_features=512, out_features=512, bias=False)
          (value): Linear(in_features=512, out_features=512, bias=True)
          (out): Linear(in_features=512, out_features=512, bias=True)
        )
        (attn_ln): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
        (mlp): Sequential(
          (0): Linear(in_features=512, out_features=2048, bias=True)
          (1): GELU(approximate='none')
          (2): Linear(in_features=2048, out_features=512, bias=True)
        )
        (mlp_ln): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
      )
    )
    (ln_post): LayerNorm((512,), eps=1e-05,

In [3]:
fcount = 0
def build_truncation_labels(word_segments, num_frames, frame_stride=0.02):
    labels = np.zeros(num_frames, dtype=np.float32)
    end_time = 0
    start_time = 0
    flag = 0
    global fcount
    for word in word_segments:
        start_time = word[0] / 1000
        end_time = word[1] / 1000
        token_end_frame = int(round(end_time / frame_stride))
        token_start_frame = int(round(start_time / frame_stride))
        alpha = 1 / (token_end_frame - token_start_frame)
        labels[token_start_frame:token_end_frame] = alpha
        if token_end_frame - token_start_frame <= 2:
            fcount += 1
            flag = 1
            break

    return torch.tensor(labels, dtype=torch.float32), flag

In [ ]:
print(whisper.__file__)

/usr/local/lib/python3.11/dist-packages/whisper/__init__.py


In [4]:
%%bash

curl -O https://us.openslr.org/resources/33/data_aishell.tgz
tar -xzf data_aishell.tgz

cd /content/data_aishell/wav

for f in *.tar.gz; do
  tar -xzf "$f"
done
echo "All done!"

All done!


  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 14.5G  100 14.5G    0     0  32.0M      0  0:07:43  0:07:43 --:--:-- 32.5M


In [5]:
import os

class AishellDataset(Dataset):
    def __init__(self, root: str, type: str, transcript_file: str = "aishell_transcript_v0.8.txt", max_samples: int = None, resample_rate: int = 16000):
        txt_path = os.path.join(root, "transcript", transcript_file)
        with open(txt_path, "r", encoding="utf-8") as f:
            self.transcripts = {
                line.strip().split(maxsplit=1)[0]: line.strip().split(maxsplit=1)[1]
                for line in f if line.strip()
            }

        wav_root = os.path.join(root, "wav", type)
        self.items = []
        for speaker in os.listdir(wav_root):
            spk_dir = os.path.join(wav_root, speaker)
            if not os.path.isdir(spk_dir):
                continue
            for fname in os.listdir(spk_dir):
                if not fname.lower().endswith(".wav"):
                    continue
                utt_id = fname[:-4]
                if utt_id in self.transcripts:
                    wav_path = os.path.join(spk_dir, fname)
                    self.items.append((wav_path, utt_id))


        if max_samples is not None:
            self.items = self.items[:max_samples]

        self.resample_rate = resample_rate

    def __len__(self):
        return len(self.items)

    def __getitem__(self, idx):
        wav_path, utt_id = self.items[idx]
        waveform, sr = torchaudio.load(wav_path)
        if sr != self.resample_rate:
            waveform = torchaudio.transforms.Resample(sr, self.resample_rate)(waveform)
            sr = self.resample_rate
        transcript = self.transcripts[utt_id]
        return waveform, wav_path, 16000, transcript

In [6]:
train_dataset = AishellDataset(root='/content/data_aishell', type='train', max_samples=100000, resample_rate=16000)

In [7]:
align_model = AutoModel(model="fa-zh", model_revision="v2.0.4", disable_pbar=True)

funasr version: 1.2.6.
Check update of funasr, and it would cost few times. You may disable it by set `disable_update=True` in AutoModel
You are using the latest version of funasr-1.2.6


2025-07-22 04:37:43,577 - modelscope - INFO - Use user-specified model revision: v2.0.4
2025-07-22 04:37:43,993 - modelscope - INFO - Got 9 files, start to download ...


Processing 9 items:   0%|          | 0.00/9.00 [00:00<?, ?it/s]


























































































































































































































































































































































































































































































































































2025-07-22 04:38:23,014 - modelscope - INFO - Download model 'iic/speech_timestamp_prediction-v1-16k-offline' successfully.


In [8]:
labels = []
encoder_outs = []
count = 0

for i in range(len(train_dataset)):
    if(count >= 200000):
      break
    waveform, wave_path, sr, transcript, *_ = train_dataset[i]
    result = align_model.generate(input=(wave_path, transcript), data_type=("sound", "text"))
    word_segments = result[0]["timestamp"]
    label, label_flag = build_truncation_labels(word_segments, 3000)
    if label_flag == 1:
      continue
    length = waveform.shape[1] / 16000
    flag = 0
    k = -1
    word_segments.insert(0, [0, 0]) #考虑开头
    #print(word_segments)
    for word in word_segments:
        k += 1
        start_offset = 0
        if(k%5 != 0):
          continue
        # if np.random.rand() < 0.2:
        #     start_offset = 0.02 * np.random.randint(0, 45)
        start = word[1] / 1000
        end = start + 1 - start_offset
        chunk = waveform[:, int(start * 16000):int(end * 16000)]
        chunk = torch.nn.functional.pad(chunk, (int(start_offset * 16000), 0))
        if end > length:
            pad_length = end - length
            chunk = torch.nn.functional.pad(chunk, (0, int(pad_length * 16000)))
            end = length
            flag = 1
        chunk = chunk[:, :16000]
        # noise = torch.randn_like(chunk) * 0.001
        # chunk = chunk + noise
        mel = whisper.log_mel_spectrogram(chunk,n_mels=80).to(device)
        with torch.no_grad():
            encoder_out = whisper_model.encoder(mel)
        encoder_out = encoder_out.squeeze(0)
        token_start = int(round(start / 0.02))
        token_end = token_start + encoder_out.shape[0] - int(start_offset/0.02)
        label_chunk = label[token_start:token_end]
        prev_pad = torch.zeros(int(start_offset / 0.02))
        if start_offset > 0:
            label_chunk = torch.cat((prev_pad, label_chunk))
        if flag == 1:
          pad_label = torch.zeros(encoder_out.shape[0] - label_chunk.shape[0])
          label_chunk = torch.cat((label_chunk, pad_label))
        if(label_chunk.shape[0] != encoder_out.shape[0]):
          print(label_chunk.shape[0], encoder_out.shape[0])
          continue
        count += 1
        labels.append(label_chunk.cpu())
        encoder_outs.append(encoder_out.cpu())
        if flag == 1:
            break
print(fcount)

262


In [9]:
class LSTMDataset(Dataset):
    def __init__(self, encoder_outs, labels, frame_stride=0.02):
        self.encoder_outs = encoder_outs
        self.frame_stride = frame_stride
        self.labels = labels

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        # Get encoder output
        encoder_out = self.encoder_outs[idx]
        encoder_out = encoder_out.to(device)
        # Get label
        label = self.labels[idx]
        label = label.to(device)
        return encoder_out, label

In [10]:
def collate_fn(batch):
    encs, labs = zip(*batch)
    encs = torch.stack(encs)  # [B, T, D]
    labs = torch.stack(labs)  # [B, T]
    return encs, labs

In [11]:
class LSTMTruncationDetector(nn.Module):
    def __init__(self, input_dim, hidden_dim=512, num_layers=2, bidirectional=True, dropout=0.3):
        super().__init__()
        self.lstm = nn.LSTM(input_dim, hidden_dim, num_layers=num_layers,
                            batch_first=True, bidirectional=bidirectional, dropout=dropout)
        self.out_proj = nn.Linear(hidden_dim * (2 if bidirectional else 1), 1)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x):  # x: [B, T, D]
        lstm_out, _ = self.lstm(x)          # [B, T, H]
        lstm_out = self.dropout(lstm_out)
        logits = self.out_proj(lstm_out)    # [B, T, 1]
        alphas = torch.sigmoid(logits).squeeze(-1)  # [B, T]
        return alphas

In [12]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class CIFTimeLoss(nn.Module):
    def __init__(self, lambda_time: float = 1.0, lambda_count: float = 1.0, lambda_blank: float = 1.0, eps: float = 1e-6):
        super().__init__()
        self.lambda_time = lambda_time
        self.lambda_count = lambda_count
        self.lambda_blank = lambda_blank
        self.eps = eps
        self.frame_reg_loss = nn.SmoothL1Loss(reduction='mean', beta=2.0)
        self.count_loss_fn = nn.SmoothL1Loss(reduction='mean', beta=0.1)

    def forward(self, alpha: torch.Tensor, true_counts: torch.Tensor, true_frames_list: list, labs: torch.Tensor, epoch: int):
        device = alpha.device
        B, T = alpha.shape

        sum_alpha = alpha.sum(dim=1)
        l_count = self.count_loss_fn(sum_alpha, true_counts)

        A = torch.cumsum(alpha, dim=1)

        l_time_sum = torch.tensor(0.0, device=device)
        valid_samples = 0
        beta = 2.0 + epoch * 0.5
        #ta = 1 / epoch

        blank_mask = (labs == 0).float()
        l_blank = (alpha * blank_mask).sum() # sum or mean

        for i in range(B):
            U_i = int(true_counts[i].item())
            if U_i <= 0:
                continue

            A_i = A[i]
            thresholds = torch.arange(1, U_i+1, device=device, dtype=A_i.dtype)

            diff = (A_i.unsqueeze(0) - thresholds.unsqueeze(1)).abs()
            penalty = (A_i.unsqueeze(0) - thresholds.unsqueeze(1)).clamp(min=0)
            # f = diff + 15 * penalty
            #weights = F.gumbel_softmax(diff, tau=ta, hard=True)
            weights = F.softmax(-beta * diff - 10 * penalty, dim=1)

            t_idx = torch.arange(T, device=device, dtype=torch.float32)
            pred_frames = (weights * t_idx).sum(dim=1)

            true_frames = true_frames_list[i].to(device).float()
            l_time_i = self.frame_reg_loss(pred_frames, true_frames)

            l_time_sum += l_time_i
            valid_samples += 1

        l_time = (l_time_sum / valid_samples) if valid_samples>0 else torch.tensor(0.0, device=device)
        loss   = self.lambda_count * l_count + self.lambda_time * l_time + self.lambda_blank * l_blank
        return loss, l_count, l_time, l_blank

In [13]:
model = LSTMTruncationDetector(input_dim=Encoder_DIM).to(device)
criterion = CIFTimeLoss(lambda_count=2.0)
optimizer = torch.optim.Adam(model.parameters(), lr=1e-4, weight_decay=1e-5)

In [14]:
def get_frame_lists(lab):
    cumsum = lab.cumsum(dim=0)
    total_tokens = int(lab.sum().item())
    fire_indices = []
    a = 1e-4
    for k in range(1, total_tokens + 1):
        idx = torch.searchsorted(cumsum, torch.tensor(k-a, device=cumsum.device)).item()
        fire_indices.append(idx)
    return torch.tensor(fire_indices, dtype=torch.long)

In [15]:
from torch.utils.data import random_split
total_dataset = LSTMDataset(encoder_outs, labels)
N = len(total_dataset)
train_size = int(0.9 * N)
test_size = N - train_size
print(f"train_size: {train_size}, test_size: {test_size}")

train_dataset, test_dataset = random_split(total_dataset, [train_size, test_size])

train_dataloader = DataLoader(train_dataset, batch_size=8, shuffle=True, collate_fn=collate_fn)
test_dataloader = DataLoader(test_dataset, batch_size=8, shuffle=False, collate_fn=collate_fn)

train_size: 180000, test_size: 20001


In [16]:
least_dev_loss = 1e9

In [17]:
from sklearn.metrics import precision_score, recall_score, f1_score, accuracy_score
num_epochs = 20
for epoch in range(1, num_epochs + 1):
    model.train()
    epoch_loss = 0.0
    dev_loss = 0.0
    dev_loss_count = 0.0
    dev_loss_frame = 0.0
    dev_loss_blank = 0.0
    all_diffs = []

    for enc, lab in train_dataloader:
        batch_size, seq_len, _ = enc.size()
        alphas = model(enc)
        true_counts = lab.sum(dim=-1).float()
        true_frames_list = [get_frame_lists(lab[i]) for i in range(batch_size)]
        loss, _, _, _ = criterion(alphas, true_counts, true_frames_list, lab, epoch)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        epoch_loss += loss.item()

    model.eval()
    for enc, lab in test_dataloader:
        with torch.no_grad():
            batch_size, seq_len, _ = enc.size()
            alphas = model(enc)
            true_counts = lab.sum(dim=-1).float()
            true_frames_list = [get_frame_lists(lab[i]) for i in range(batch_size)]
            pred_counts = alphas.sum(dim=-1).float()
            diff = (pred_counts - true_counts).abs().cpu().numpy().tolist()
            loss, loss_count, loss_frame, loss_blank = criterion(alphas, true_counts, true_frames_list, lab, epoch)
            dev_loss += loss.item()
            dev_loss_count += loss_count.item()
            dev_loss_frame += loss_frame.item()
            dev_loss_blank += loss_blank.item()
            all_diffs.extend(diff)
    epoch_loss /= len(train_dataloader)
    dev_loss /= len(test_dataloader)
    dev_loss_count /= len(test_dataloader)
    dev_loss_frame /= len(test_dataloader)
    dev_loss_blank /= len(test_dataloader)
    avg_diff = np.mean(all_diffs)
    print(f"Epoch {epoch}/{num_epochs}, Loss: {epoch_loss:.4f}, Avg Diff: {avg_diff:.4f}, Dev Loss: {dev_loss:.4f}")
    print(f"  Count Loss: {dev_loss_count:.4f}, Frame Loss: {dev_loss_frame:.4f}, Blank Loss: {dev_loss_blank:.4f}")
    if dev_loss < least_dev_loss:
        least_dev_loss = dev_loss
        torch.save(model.state_dict(), f"model.pth")
        print("Model saved!")

Epoch 1/20, Loss: 1.9886, Avg Diff: 0.1598, Dev Loss: 1.4963
  Count Loss: 0.1216, Frame Loss: 1.1856, Blank Loss: 0.0675
Model saved!
Epoch 2/20, Loss: 1.1896, Avg Diff: 0.1443, Dev Loss: 1.1306
  Count Loss: 0.1070, Frame Loss: 0.8397, Blank Loss: 0.0769
Model saved!
Epoch 3/20, Loss: 0.9857, Avg Diff: 0.1440, Dev Loss: 0.9716
  Count Loss: 0.1069, Frame Loss: 0.6970, Blank Loss: 0.0609
Model saved!
Epoch 4/20, Loss: 0.8867, Avg Diff: 0.1396, Dev Loss: 0.8981
  Count Loss: 0.1021, Frame Loss: 0.6369, Blank Loss: 0.0570
Model saved!
Epoch 5/20, Loss: 0.8300, Avg Diff: 0.1497, Dev Loss: 0.9210
  Count Loss: 0.1100, Frame Loss: 0.6464, Blank Loss: 0.0546
Epoch 6/20, Loss: 0.7941, Avg Diff: 0.1369, Dev Loss: 0.8387
  Count Loss: 0.0985, Frame Loss: 0.5867, Blank Loss: 0.0549
Model saved!
Epoch 7/20, Loss: 0.7659, Avg Diff: 0.1302, Dev Loss: 0.7985
  Count Loss: 0.0933, Frame Loss: 0.5508, Blank Loss: 0.0613
Model saved!
Epoch 8/20, Loss: 0.7377, Avg Diff: 0.1308, Dev Loss: 0.7942
  Count

KeyboardInterrupt: 

In [18]:
def get_token_frame_intervals(alphas: torch.Tensor, threshold: float = 1.0):
    B, T = alphas.size()
    token_counts = []
    intervals = []

    for b in range(B):
        integrate = 0.0
        prev_fire = -1
        samps = alphas[b].tolist()

        this_intervals = []
        for t, a in enumerate(samps):
            integrate += a
            if integrate >= threshold:
                start = prev_fire + 1
                end   = t
                this_intervals.append((start, end))
                prev_fire = t
                integrate -= threshold

        token_counts.append(len(this_intervals))
        intervals.append(this_intervals)

    return token_counts, intervals

In [19]:
def get_token_frame_intervals_single(alphas: torch.Tensor, threshold: float = 1.0):
    integrate = 0.0
    prev_fire = -1
    samps = alphas.tolist()

    intervals = []
    for t, a in enumerate(samps):
        integrate += a
        if integrate >= threshold:
            start = prev_fire + 1
            end = t
            intervals.append((start, end))
            prev_fire = t
            integrate -= threshold

    token_count = len(intervals)
    return token_count, intervals

In [ ]:
# def resize(alphas: torch.Tensor,
#            target_lengths: torch.Tensor,
#            noise: float = 0.0,
#            threshold: float = 1.0,
#            max_iter: int = 20):
#     device = alphas.device
#     B, T = alphas.size()

#     orig_sums = alphas.sum(dim=-1)

#     num = float(target_lengths)
#     if noise > 0:
#         num = num + noise * torch.rand_like(num)

#     scale = (num / (orig_sums + 1e-8)).unsqueeze(1)
#     resized = alphas * scale

#     for _ in range(max_iter):
#         mask_exceed = resized > threshold
#         if not mask_exceed.any():
#             break
#         for b in torch.unique(mask_exceed.nonzero()[:,0]):
#             row = resized[b]
#             mask = row.ne(0).float()
#             mean_val = 0.5 * row.sum() / (mask.sum() + 1e-8)
#             resized[b] = row * 0.5 + mean_val * mask

#     return resized, orig_sums

In [20]:
def resize_single(alphas: torch.Tensor,
              target_length,
              noise: float = 0.0,
              threshold: float = 1.0,
              max_iter: int = 20):

    device = alphas.device
    orig_sum = alphas.sum()

    if isinstance(target_length, torch.Tensor):
        num = target_length.to(device)
    else:
        num = torch.tensor(float(target_length), device=device)

    if noise > 0:
        num = num + noise * torch.rand((), device=device)

    scale = num / (orig_sum + 1e-8)
    resized = alphas * scale

    for _ in range(max_iter):
        mask_exceed = resized > threshold
        if not mask_exceed.any():
            break

        row = resized
        mask = row.ne(0).float()
        mean_val = 0.5 * row.sum() / (mask.sum() + 1e-8)
        resized = row * 0.5 + mean_val * mask

    return resized, orig_sum

In [21]:
def truncate_alphas(alphas: torch.Tensor, threshold: float = 1.0) -> torch.Tensor:
    B, T = alphas.shape
    device = alphas.device

    truncated = alphas.clone()

    for b in range(B):
        row = alphas[b]
        A = torch.cumsum(row, dim=0)

        total = A[-1].item()
        K = torch.floor(torch.tensor(total / threshold, device=device)) * threshold
        if K < threshold:
            left = 0
            continue
        left = total - K
        idx = (A >= K).nonzero(as_tuple=False)
        t_last = idx[0, 0].item()

        if t_last + 1 < T:
            truncated[b, t_last+1 :] = 0.0

    return truncated, left

In [22]:
def Judge_truncate(pred_interval, left, alpha):
    if len(pred_interval) == 0:
        return False, alpha
    last_interval = pred_interval[-1]
    if last_interval[1] == 49 and len(pred_interval) != 1:
        alpha[last_interval[0]:] = 0.0
        return True, alpha
    if last_interval[1] <= 45 and left >= 0.09:
        return True, alpha
    return False, alpha

In [23]:
def train_frame(alpha, count):
    U_i = int(count)
    if U_i <= 0:
        return []
    A_i = torch.cumsum(alpha, dim=0)
    thresholds = torch.arange(1, U_i+1, device=device, dtype=A_i.dtype)

    diff = (A_i.unsqueeze(0) - thresholds.unsqueeze(1)).abs()
    penalty = (A_i.unsqueeze(0) - thresholds.unsqueeze(1)).clamp(min=0)
    f = diff
    weights = F.softmax(-10 * f - 10 * penalty, dim=1)

    t_idx = torch.arange(50, device=device, dtype=torch.float32)
    pred_frames = (weights * t_idx).sum(dim=1)
    return pred_frames

In [24]:
model.load_state_dict(torch.load("model.pth"))

<All keys matched successfully>

In [25]:
model.eval()
ct1 = 0
ct2 = 0
diffs = []
criterion = CIFTimeLoss()
for enc, lab in test_dataloader:
    with torch.no_grad():
        batch_size, seq_len, _ = enc.size()
        alphas = model(enc)
        true_counts = lab.sum(dim=-1).float()
        for b in range(enc.size(0)):
            true_count = true_counts[b]
            a_sum = alphas[b].sum()
            train_frames = train_frame(alphas[b], true_count)
            true_frames = get_frame_lists(lab[b])
            pred_token_count, pred_interval = get_token_frame_intervals_single(alphas[b], threshold=0.99)
            # if true_count != 1:
            #   continue
            print(f"  true_counts: {true_counts[b]}, a_sum: {a_sum}")
            print(f"  train_frames: {train_frames}")
            print(f"  true_frames: {true_frames}")
            print(f"  pred_interval: {pred_interval}")
            print(alphas[b][0:10])


流式输出内容被截断，只能显示最后 5000 行内容。
        2.7228e-05, 1.1960e-06, 9.2186e-08, 5.9985e-08], device='cuda:0')
  true_counts: 4.0, a_sum: 4.5348944664001465
  train_frames: tensor([ 7.2426, 15.2214, 24.5235, 36.5662], device='cuda:0')
  true_frames: tensor([ 9, 17, 25, 46])
  pred_interval: [(0, 5), (6, 12), (13, 21), (22, 37)]
tensor([5.1924e-05, 3.7957e-05, 5.4056e-04, 2.8837e-02, 4.9679e-01, 4.8896e-01,
        3.4401e-03, 1.7567e-03, 4.9241e-04, 4.0862e-03], device='cuda:0')
  true_counts: 3.384615421295166, a_sum: 3.5772154331207275
  train_frames: tensor([11.0654, 27.5771, 41.4971], device='cuda:0')
  true_frames: tensor([11, 27, 41])
  pred_interval: [(0, 5), (6, 22), (23, 36)]
tensor([4.4921e-06, 4.2422e-05, 3.2735e-04, 7.6103e-02, 4.9361e-01, 4.4972e-01,
        5.5469e-03, 1.3721e-02, 2.7817e-02, 3.0062e-03], device='cuda:0')
  true_counts: 3.500000238418579, a_sum: 3.490102767944336
  train_frames: tensor([13.4726, 25.8965, 43.1124], device='cuda:0')
  true_frames: tensor([15, 27, 43]

In [ ]:
np.max(diffs)